In [7]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [3]:
import tfx
import os
from tfx.components import CsvExampleGen

In [2]:
import tfx
print('TFX version: {}'.format(tfx.__version__))

TFX version: 1.8.0


In [10]:
_root = r'C:\Users\skumar\tfx_custom_component_pipeline'
_tfx_root = os.path.join(_root, 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines')

context = InteractiveContext(pipeline_root=_tfx_root)

# 1st component (CSVexampleGen) 

In [11]:
# 1st standard tfx component to get the data from CSV file
_data_root = os.path.join(_root, 'data')
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)

In [12]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [27]:
examples=example_gen.outputs['examples']

In [38]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]


In [45]:
# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 1 records and decode them.
for tfrecord in dataset:
    result = tf.io.serialize_tensor(tfrecord).numpy()
    result = tf.io.parse_tensor(result,out_type=).pandas()
    result

TypeError: parse_tensor() missing 1 required positional argument: 'out_type'

In [40]:
serialized_example

b'\n\xfc\x02\n\x1d\n\tworkclass\x12\x10\n\x0e\n\x0cSelf-emp-inc\n\x0c\n\x03age\x12\x05\x1a\x03\n\x014\n!\n\noccupation\x12\x13\n\x11\n\x0fExec-managerial\n\x11\n\x04race\x12\t\n\x07\n\x05White\n\x18\n\teducation\x12\x0b\n\t\n\x07HS-grad\n\x18\n\x0crelationship\x12\x08\n\x06\n\x04Wife\n(\n\x0emarital-status\x12\x16\n\x14\n\x12Married-civ-spouse\n\x11\n\x06fnlwgt\x12\x07\x1a\x05\n\x03\xb7\xc9\x11\n#\n\x0enative-country\x12\x11\n\x0f\n\rUnited-States\n\x16\n\x0ccapital-gain\x12\x06\x1a\x04\n\x02\xb0u\n\x15\n\x0ccapital-loss\x12\x05\x1a\x03\n\x01\x00\n\x11\n\x03sex\x12\n\n\x08\n\x06Female\n\x0e\n\x05label\x12\x05\x1a\x03\n\x01\x01\n\x17\n\x0ehours-per-week\x12\x05\x1a\x03\n\x01(\n\x16\n\reducation-num\x12\x05\x1a\x03\n\x01\t'

# 2nd component (StatisticsGen)

In [15]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

In [16]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

# 3rd component (SchemaGen)

In [18]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

In [19]:
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

# 4th Component (Custom component)

Sklearn tranformer pipeline to tranform the data 

In [ ]:
class OutlierRemover(BaseEstimator,TransformerMixin):
    def __init__(self,factor=1.5):
        self.factor = factor

    def outlier_removal(self,X,y=None):
        X = pd.Series(X).copy()
        q1 = X.quantile(0.25)
        q3 = X.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (self.factor * iqr)
        upper_bound = q3 + (self.factor * iqr)
        X.loc[((X < lower_bound) | (X > upper_bound))] = np.nan 
        return pd.Series(X)

    def fit(self,X,y=None):
        return self

    def transform(self,X,y=None):
        return X.apply(self.outlier_removal)

In [26]:
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

import tfx.v1 as tfx
from tfx.dsl.component.experimental.decorators import component

@component
def MyTransformerComponent(
    input_data: tfx.dsl.components.InputArtifact[tfx.types.standard_artifacts.Examples],
    tranformed_data: tfx.dsl.components.OutputArtifact[tfx.types.standard_artifacts.Model],
    ):
    
    
    

    
    
    

      return {
        'loss': model_obj.loss,
        'accuracy': model_obj.accuracy}

In [30]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])

In [31]:
for element in dataset:
    print(element)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


In [32]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [33]:
import numpy as np
n = np.array([1, 2, 3])

In [34]:
n

array([1, 2, 3])

In [35]:
n.shape

(3,)

In [41]:
tensor = tf.convert_to_tensor(np.random.random(size=(5,5,2)))
result = tf.io.serialize_tensor(tensor).numpy()
result = tf.io.parse_tensor(result, out_type=tf.float64).numpy()
result

array([[[0.47117478, 0.81892941],
        [0.25322652, 0.15729247],
        [0.68286599, 0.75424578],
        [0.60065458, 0.23097998],
        [0.36613359, 0.74245681]],

       [[0.35393807, 0.64264212],
        [0.62543307, 0.45144994],
        [0.24365124, 0.32711232],
        [0.54324094, 0.10568459],
        [0.20575864, 0.36591038]],

       [[0.18587465, 0.67404645],
        [0.14104946, 0.97798931],
        [0.83200944, 0.92524225],
        [0.72004959, 0.0906714 ],
        [0.71377073, 0.6831979 ]],

       [[0.02567051, 0.42754599],
        [0.61766427, 0.66810895],
        [0.8577751 , 0.88468739],
        [0.45911694, 0.88877571],
        [0.69557859, 0.78071326]],

       [[0.21227571, 0.3304915 ],
        [0.53018667, 0.38288756],
        [0.18807291, 0.8703778 ],
        [0.72667089, 0.28930625],
        [0.97571797, 0.40797105]]])

In [42]:
type(tensor)

tensorflow.python.framework.ops.EagerTensor